In [1]:
from queue import Queue
import time

In [103]:
def read_input(filename, insert = False):
    with open(filename, 'r') as f:
        d = [s.rstrip() for s in f.readlines()]
        d = [[s for s in ss] for ss in d]
    ncol = len(d[0])
    nrow = len(d)
    y = ncol // 2
    x = nrow // 2
    if insert:
        for (dx, dy) in [(0, 0), (-1, 0), (1, 0), (0, -1), (0, 1)]:
            d[x + dx][y + dy] = '#'
        for (dx, dy) in [(-1, -1), (-1, 1), (1, -1), (1, 1)]:
            d[x + dx][y + dy] = '@'
    d1 = {(i, j): d[i][j] for i in range(nrow // 2 + 1) for j in range(ncol // 2 + 1)}
    d2 = {(i, j - ncol // 2): d[i][j] for i in range(nrow // 2 + 1) for j in range(ncol // 2, ncol)}
    d3 = {(i - nrow // 2, j): d[i][j] for i in range(nrow // 2, nrow) for j in range(ncol // 2 + 1)}
    d4 = {(i - nrow // 2, j - ncol // 2): d[i][j] for i in range(nrow // 2, nrow) for j in range(ncol // 2, ncol)}
    return d1, d2, d3, d4

In [104]:
d1, d2, d3, d4 = read_input('18_testinput4.txt')

In [48]:
def get_adjacent(p, tunnel_map, all_keys):
    (x, y), keys = p
    doors = [k.upper() for k in keys]
    adj = []
    for p_a in [(x - 1, y), (x + 1, y), (x, y - 1), (x, y + 1)]:
        if tunnel_map[p_a] in ['.', '@'] + doors + all_keys:
            adj.append((p_a, p[1]))
    return adj

def bfs(tunnel_map, p_start, all_keys):
    q = Queue()
    q.put(p_start)
    explored = set()
    dist = {p_start: 0}
    while not q.empty():
        v = q.get()
        xy, keys = v
        if tunnel_map[xy] in all_keys:
            if not tunnel_map[xy] in keys:
                v_old = v
                v = (xy, keys | frozenset(tunnel_map[xy]))
                dist[v] = dist[v_old]
        if set(v[1]) == set(all_keys):
            return dist[v]
        for w in get_adjacent(v, tunnel_map, all_keys):
            if not w in explored:
                explored.add(w)
                dist[w] = dist[v] + 1
                q.put(w)
    return -1


In [49]:
def runit(filename, alg = bfs):
    tunnel_map = read_input(filename)
    all_keys = [c for c in list(set(tunnel_map.values()) - set(['#', '.', '@'])) if c.islower()]
    p_start = get_key(tunnel_map, '@')
    t0 = time.time()
    dist = alg(tunnel_map, (p_start, frozenset()), all_keys)
    dt = time.time() - t0
    res = {'time': dt, 'distance': dist}
    return res

In [50]:
res = runit('18_input.txt')
res

{'time': 5.620743989944458, 'distance': 4350}